In [1]:
import json

In [13]:
def load_annot(fname):
    elem_list = []
    with open(fname) as f:
        lines = f.readlines()
        for line in lines:
            line = line.replace("]]","]")
            elem = json.loads(line.strip())
            
            elem_list.append(elem)
    return elem_list

In [14]:
items = load_annot('../annotation/train.json')

[{'docId': 'AFP_ENG_20100128.0429', 'sentiments': [{'strength': '', 'sentiment': 'None', 'holder': 'Darlene Etienne', 'target': 'Darlene Etienne'}, {'strength': '', 'sentiment': 'Negative', 'holder': 'Orcel', 'target': 'Darlene Etienne'}]}, {'docId': 'AFP_ENG_20100529.0157', 'sentiments': [{'strength': '', 'sentiment': 'Negative', 'holder': 'Khamenei', 'target': 'Rafsanjani'}, {'strength': '', 'sentiment': 'None', 'holder': 'Rafsanjani', 'target': 'Rafsanjani'}, {'strength': '', 'sentiment': 'Invalid', 'holder': 'Ahmadinejad', 'target': 'Khamenei'}, {'strength': '', 'sentiment': 'Negative', 'holder': 'Rafsanjani', 'target': 'Khamenei'}, {'strength': '', 'sentiment': 'Negative', 'holder': 'Ahmadinejad', 'target': 'Rafsanjani'}, {'strength': '', 'sentiment': 'None', 'holder': 'Khamenei', 'target': 'Khamenei'}]}, {'docId': 'NYT_ENG_20100412.0128', 'sentiments': [{'strength': '', 'sentiment': 'Positive', 'holder': 'Miller', 'target': 'Schauer'}, {'strength': '', 'sentiment': 'None', 'holde